In [722]:
import pylab as py
import numpy as np
import matplotlib.pyplot as plt
from math import pi
from scipy.spatial.distance import cdist
from scipy.stats import multivariate_normal
import numpy.random
from numpy.linalg import inv

#global variables
random_xs = []
y_values = []
w = [-1.3, 0.5]

#Define prior assumptions
identity_matrix = np.identity(2)
prior_w_mean = np.array([0,0])
prior_w_cov = identity_matrix
prior_w = multivariate_normal(prior_w_mean.flatten(), prior_w_cov)

#Plot contour
x = np.linspace(-1, 1, 100)
x1p, x2p = np.meshgrid(x,x)
x1p_flat = x1p.flatten()
x2p_flat = x2p.flatten()
pos = np.vstack((x1p_flat, x2p_flat))
pos = pos.T
Z = prior_w.pdf(pos)
Z = Z.reshape(100,100)
fig = plt.figure()
ax = fig.add_subplot(111)
prior_w_contour = ax.contour(x1p, x2p, Z, 3, colors='k')

#Generate x and y values
def get_random_xs(n):
    for i in range(0, n):
        index = np.random.randint(0, len(x)-1)
        while x[index] in random_xs:
            index = np.random.randint(1, len(x)-1)
            print("looping")
        random_x = x[index]
        random_xs.append(random_x)
    return random_xs

def generate_y_value(random_xs):
    for x in random_xs:
        y = x*w[0] + w[1]
        y_values.append(y)
    Y = np.array([y_values])
    Y = Y.T
    print("here is YYY")
    print(Y)
    return Y

#Turn random_xs into matrix with ones
def generate_x_and_ones_vector(random_xs):
    print("random_xs")
    print(random_xs)
    arr_x = np.array([random_xs])
    arr_x = arr_x.T
    print("arr_x")
    print(arr_x)
    ones = np.array([np.ones(len(random_xs))])
    ones = ones.T
    print("x_and_ones")
    x_and_ones = np.concatenate((arr_x, ones), axis=1)
    print(x_and_ones)
    #x_and_ones = np.array([x_and_ones])
    return x_and_ones

#Generate an x and turn into (1,2) matrix     
xs = get_random_xs(10)
X = generate_x_and_ones_vector(xs)
Y = generate_y_value(xs)


#Calculate posterior (formulas from GaussianIDentities.pdf end of page 4)
posterior_cov = (1/(0.3))*np.dot(X.T, X) + inv(identity_matrix)  #0.3 or 0.3**2?
posterior_mean = (1/(0.3))*np.dot(inv(posterior_cov), np.dot(X.T, Y))
print("cov")
print(posterior_cov)
print("mean")
print(posterior_mean)
posterior_w = multivariate_normal(posterior_mean.flatten(), inv(posterior_cov))

#Plot contour for posterior of w given single random datapoint
Z_posterior = posterior_w.pdf(pos)
Z_posterior = Z_posterior.reshape(100,100)
posterior_w_contour = ax.contour(x1p, x2p, Z_posterior, 3, colors='k')


#plt.scatter(random_x, y, color='g')
#plt.plot(random_x, y, 'r')
#y = x[:]*w[0] + w[1] + e[:,0]
plt.axis([-1.5, 1.5, -1.5, ])
plt.show()

1.43232323232
